In [ ]:
!pip install scrapegraphai pandas beautifulsoup4 requests


In [ ]:
!pip install scrapegraphai openai pandas nest_asyncio


In [1]:
pip install requests beautifulsoup4 pandas


Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
from urllib.parse import urljoin

# === Setup ===
base_url = "https://highape.com"
event_listing_url = f"{base_url}/bangalore/all-events"

# === Get Event Links ===
res = requests.get(event_listing_url)
soup = BeautifulSoup(res.text, "html.parser")
event_links = [urljoin(base_url, a["href"]) for a in soup.select("a.event-box")[:5]]

# === Helper ===
def safe_text(elem):
    return elem.get_text(strip=True) if elem else ""

# === Parse Each Event ===
def parse_event_page(url):
    r = requests.get(url)
    p = BeautifulSoup(r.text, "html.parser")
    event = {
        "eventName": safe_text(p.select_one("h1.event-title")),
        "eventDescription": str(p.select_one(".event-description") or ""),
        "eventDateAndTime": "",
        "eventDuration": None,
        "venue": {
            "venueName": safe_text(p.select_one(".venue-name")),
            "locality": "", "address": "", "city": "", "state": "", "zipcode": "",
            "geolocation": {"lat": None, "lon": None},
            "layout": ""
        },
        "highlightImageLinks": [],
        "galleryImageLinks": [],
        "ticketAmount": None,
        "ticketLink": url,
        "supportedLanguages": [],
        "category": "", "subCategory": "",
        "eventType": "Event", "eventFeatures": {}, "artists": [],
        "sharableEventOgImageLink": "", "attendeesCount": 0, "likesCount": 0,
        "joinChatDetails": {"joinChatLink": "", "provider": "", "isEnabled": False},
        "policyAndConditions": [], "frequentlyAskedQuestions": []
    }

    # Extract date
    dt_raw = safe_text(p.select_one(".event-details p"))
    match = re.search(r"(\d{1,2} \w+ \d{4} \| \d{1,2}:\d{2} [APMapm]{2})", dt_raw)
    if match:
        try:
            dt = datetime.strptime(match.group(1), "%d %B %Y | %I:%M %p")
            event["eventDateAndTime"] = dt.isoformat()
        except:
            pass

    img = p.select_one("div.event-img img")
    if img and img.get("src"):
        full_img = urljoin(base_url, img["src"])
        event["highlightImageLinks"].append(full_img)
        event["sharableEventOgImageLink"] = full_img

    return event

# === Run ===
events_data = [parse_event_page(link) for link in event_links]
df = pd.json_normalize(events_data)
print(df.head())


Empty DataFrame
Columns: []
Index: []


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
from urllib.parse import urljoin

base_url = "https://highape.com"
event_listing_url = f"{base_url}/bangalore/all-events"

# Step 1: Check if events are being found
res = requests.get(event_listing_url)
soup = BeautifulSoup(res.text, "html.parser")

# Try multiple selectors to get event URLs
event_links = []

for a in soup.select("a.event-box[href]"):
    href = a["href"]
    if href and "/event/" in href:
        event_links.append(urljoin(base_url, href))

print("✅ Event links found:")
print(event_links[:5])  # Only top 5

# Step 2: If event links are empty, print the raw HTML
if not event_links:
    print("❌ No event links found. Dumping sample HTML:")
    print(soup.prettify()[:2000])  # Print a small portion for inspection

# Step 3: Proceed only if links exist
def safe_text(elem):
    return elem.get_text(strip=True) if elem else ""

def parse_event_page(url):
    print(f"Scraping: {url}")
    r = requests.get(url)
    p = BeautifulSoup(r.text, "html.parser")
    
    # Debug raw title to check if it's found
    raw_title = p.select_one("h1.event-title")
    print("Event Title Found:" if raw_title else "❌ No Event Title Found")

    return {
        "eventName": safe_text(raw_title),
        "eventDescription": str(p.select_one(".event-description") or ""),
        "ticketLink": url
    }

if event_links:
    events_data = [parse_event_page(link) for link in event_links[:5]]
    df = pd.DataFrame(events_data)
    print("✅ Final DataFrame:")
    print(df)
else:
    print("⚠️ No events to scrape.")


✅ Event links found:
[]
❌ No event links found. Dumping sample HTML:
<!DOCTYPE html>
<html lang="en-in">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="https://highape.com/bangalore/all-events" property="og:url">
   <meta content="website" property="og:type">
    <meta content="947832055339051" property="fb:app_id">
     <meta content="https://highape.com/images/Highape_og.jpg" property="og:image"/>
     <link href="https://highape.com/bangalore/all-events?page=1" id="next_url" rel="next"/>
     <title>
      252 Upcoming Events and Parties in Bangalore | HighApe
     </title>
     <meta content="Select from the best  events in bangalore, for a wonderful evening. Select from multiple categories like DJ nights, Concerts, Band Performance, ladies night, comedy shows and more for unlimited fun." name="description"/>
     <meta content="252 Upcoming Events an

In [7]:
import nest_asyncio
import asyncio
import re
import uuid
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

nest_asyncio.apply()

# === Utility: Extract geolocation ===
def extract_geolocation_from_script(script_text):
    match = re.search(r'"lat"\s*:\s*([0-9.]+),\s*"lng"\s*:\s*([0-9.]+)', script_text)
    if match:
        return float(match.group(1)), float(match.group(2))
    return "", ""

# === Scraper: Parse individual event page ===
async def parse_event_page(page, url):
    await page.goto(url, timeout=60000)
    content = await page.content()
    soup = BeautifulSoup(content, "html.parser")

    # Extract fields
    try:
        name = soup.select_one("h1.headtext, h1.event-title")
        description_block = soup.select_one("div#event_details") or soup.select_one("div.about-event")
        full_desc = str(description_block) if description_block else ""

        address_block = soup.find(string=re.compile(r"HIGHAPE VENUE", re.I))
        address = address_block.find_next("p").get_text(strip=True) if address_block else ""

        ticket_link = url
        ticket_price = None
        price_text = soup.find(string=re.compile(r"Cover charges|₹|Rs", re.I))
        if price_text:
            match = re.search(r"[₹Rs\. ]+(\d+)", price_text)
            if match:
                ticket_price = int(match.group(1))

        script_with_geo = soup.find("script", string=re.compile("lat.*lng"))
        lat, lon = extract_geolocation_from_script(script_with_geo.string if script_with_geo else "")

        return {
            "eventName": name.get_text(strip=True) if name else "",
            "eventDescription": full_desc,
            "eventDateAndTime": "",  # Placeholder
            "eventDuration": "",     # Placeholder
            "venue": {
                "venueName": "",  # Not clearly mentioned
                "locality": "",   # Could be parsed later
                "address": address,
                "city": "",       # Derive from address if needed
                "state": "",
                "zipcode": "",
                "geolocation": {
                    "lat": lat,
                    "lon": lon
                },
                "layout": ""
            },
            "highlightImageLinks": [],  # Placeholder
            "galleryImageLinks": [],
            "ticketAmount": ticket_price,
            "ticketLink": ticket_link,
            "supportedLanguages": [],
            "category": "",
            "subCategory": "",
            "eventType": "Event",
            "eventFeatures": {},
            "artists": [],
            "sharableEventOgImageLink": "",
            "attendeesCount": 0,
            "likesCount": 0,
            "joinChatDetails": {
                "joinChatLink": "",
                "provider": "",
                "isEnabled": False
            },
            "policyAndConditions": [],
            "frequentlyAskedQuestions": []
        }

    except Exception as e:
        print(f"❌ Failed to parse {url} — {e}")
        return {}

# === Extract first 5 event links from all-events page ===
async def scrape_highape_events():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto("https://highape.com/bangalore/all-events", timeout=60000)
        await page.wait_for_timeout(3000)
        content = await page.content()
        soup = BeautifulSoup(content, "html.parser")

        # Find event URLs
        cards = soup.select("div.event-card a")
        links = ["https://highape.com" + a["href"] for a in cards if a.has_attr("href")]
        links = list(dict.fromkeys(links))[:5]  # Remove duplicates, limit to 5

        print("✅ Event links found:")
        print(links)

        results = []
        for link in links:
            result = await parse_event_page(await browser.new_page(), link)
            if result:
                results.append(result)

        await browser.close()

        # Convert to DataFrame
        return pd.DataFrame(results)

# === Run & Display in Jupyter ===
df = await scrape_highape_events()
df.head()


✅ Event links found:
[]


""


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from urllib.parse import urlparse
from datetime import datetime

# === Helper ===
def safe_text(elem):
    return elem.get_text(strip=True) if elem else ""

def parse_event_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    event = {
        "eventName": "",
        "eventDescription": "",
        "eventDateAndTime": "",
        "eventDuration": "",
        "venue": {
            "venueName": "",
            "locality": "",
            "address": "",
            "city": "",
            "state": "",
            "zipcode": "",
            "geolocation": {
                "lat": "",
                "lon": ""
            },
            "layout": ""
        },
        "highlightImageLinks": [],
        "galleryImageLinks": [],
        "ticketAmount": "",
        "ticketLink": url,
        "supportedLanguages": [],
        "category": "",
        "subCategory": "",
        "eventType": "Event",
        "eventFeatures": {},
        "artists": [],
        "sharableEventOgImageLink": "",
        "attendeesCount": 0,
        "likesCount": 0,
        "joinChatDetails": {
            "joinChatLink": "",
            "provider": "",
            "isEnabled": False
        },
        "policyAndConditions": [],
        "frequentlyAskedQuestions": []
    }

    # === Extract Fields ===
    event["eventName"] = safe_text(soup.select_one("h1.headtext, h1.event-title"))
    desc_block = soup.select_one("#event_details, .about-event")
    if desc_block:
        event["eventDescription"] = str(desc_block)

    # === Date/Time ===
    dt_text = safe_text(soup.select_one(".date-section"))
    dt_match = re.search(r"(\d{1,2} \w+ \d{4}).*?(\d{1,2}:\d{2} [APMapm]{2})", dt_text)
    if dt_match:
        try:
            parsed_dt = datetime.strptime(f"{dt_match.group(1)} {dt_match.group(2)}", "%d %B %Y %I:%M %p")
            event["eventDateAndTime"] = parsed_dt.isoformat()
        except Exception:
            pass

    # === Venue & Address ===
    venue_parts = soup.select(".event-venue-name")
    event["venue"]["venueName"] = safe_text(venue_parts[0]) if venue_parts else ""
    addr = soup.find("div", class_="event-venue-address")
    if addr:
        address_text = safe_text(addr)
        event["venue"]["address"] = address_text
        # Try to extract pincode
        pincode_match = re.search(r"(\d{6})", address_text)
        if pincode_match:
            event["venue"]["zipcode"] = pincode_match.group(1)
        # Attempt basic locality/city parsing
        parts = address_text.split(",")
        if len(parts) >= 3:
            event["venue"]["locality"] = parts[-3].strip()
            event["venue"]["city"] = parts[-2].strip()
            event["venue"]["state"] = parts[-1].strip()

    # === Image Links ===
    img = soup.select_one("meta[property='og:image']")
    if img:
        img_url = img["content"]
        event["highlightImageLinks"] = [img_url]
        event["sharableEventOgImageLink"] = img_url

    # === Ticket Info ===
    ticket_section = soup.find(string=re.compile("₹|Rs|INR", re.I))
    if ticket_section:
        ticket_match = re.search(r"[₹Rs\. ]+(\d+)", ticket_section)
        if ticket_match:
            event["ticketAmount"] = int(ticket_match.group(1))

    return event

# === Test with 1 or more URLs ===
event_urls = [
    "https://highape.com/bangalore/events/match-me-please-5ewaeRxjwE"
]

events = [parse_event_page(url) for url in event_urls]
df = pd.json_normalize(events)
print(df.head())


  eventName eventDescription eventDateAndTime eventDuration  \
0                                                             

                                 highlightImageLinks galleryImageLinks  \
0  [https://res.cloudinary.com/https-highape-com/...                []   

   ticketAmount                                         ticketLink  \
0            17  https://highape.com/bangalore/events/match-me-...   

  supportedLanguages category  ... venue.address venue.city venue.state  \
0                 []           ...                                        

  venue.zipcode  venue.geolocation.lat  venue.geolocation.lon venue.layout  \
0                                                                            

  joinChatDetails.joinChatLink joinChatDetails.provider  \
0                                                         

  joinChatDetails.isEnabled  
0                     False  

[1 rows x 30 columns]


In [13]:
import nest_asyncio
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup
import pandas as pd
import re
from IPython.display import display
import html

nest_asyncio.apply()

# === Step 1: Load HighApe event detail page via pyppeteer ===
async def get_page_content(url):
    browser = await launch(headless=True, args=['--no-sandbox'])
    page = await browser.newPage()
    await page.goto(url, {'waitUntil': 'networkidle2'})
    content = await page.content()
    await browser.close()
    return content

# === Step 2: Parse key fields from the event detail page ===
def parse_event_page(html_content):
    soup = BeautifulSoup(html_content, "html.parser")

    def clean_text(el):
        return html.unescape(el.get_text(strip=True)) if el else ""

    def extract_json_ld():
        script_tag = soup.find("script", type="application/ld+json")
        if not script_tag:
            return {}
        try:
            import json
            return json.loads(script_tag.string)
        except:
            return {}

    ld_json = extract_json_ld()

    # Extract metadata from JSON-LD and fallback to soup if needed
    event = {
        "eventName": ld_json.get("name", ""),
        "eventDescription": BeautifulSoup(ld_json.get("description", ""), "html.parser").prettify(),
        "eventDateAndTime": ld_json.get("startDate", ""),
        "eventDuration": "",  # Duration not always available
        "venue": {
            "venueName": ld_json.get("location", {}).get("name", ""),
            "locality": "",
            "address": ld_json.get("location", {}).get("address", {}).get("streetAddress", ""),
            "city": ld_json.get("location", {}).get("address", {}).get("addressLocality", ""),
            "state": ld_json.get("location", {}).get("address", {}).get("addressRegion", ""),
            "zipcode": ld_json.get("location", {}).get("address", {}).get("postalCode", ""),
            "geolocation": {
                "lat": ld_json.get("location", {}).get("geo", {}).get("latitude", ""),
                "lon": ld_json.get("location", {}).get("geo", {}).get("longitude", "")
            },
            "layout": ""
        },
        "highlightImageLinks": [ld_json.get("image", "")] if ld_json.get("image") else [],
        "galleryImageLinks": [],
        "ticketAmount": "",
        "ticketLink": "",
        "supportedLanguages": [],
        "category": "",
        "subCategory": "",
        "eventType": "Event",
        "eventFeatures": {},
        "artists": [],
        "sharableEventOgImageLink": ld_json.get("image", ""),
        "attendeesCount": "",
        "likesCount": "",
        "joinChatDetails": {
            "joinChatLink": "",
            "provider": "",
            "isEnabled": False
        },
        "policyAndConditions": [],
        "frequentlyAskedQuestions": []
    }

    return event

# === Step 3: Run everything and show in scrollable DataFrame ===
async def scrape_event_and_display(url):
    html_content = await get_page_content(url)
    event_data = parse_event_page(html_content)
    df = pd.json_normalize(event_data, sep='.')
    pd.set_option('display.max_colwidth', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.expand_frame_repr', False)
    display(df)

# === Step 4: Provide your test URL ===
event_url = "https://highape.com/bangalore/events/match-me-please-5ewaeRxjwE?utm_source=allevents_list"
await scrape_event_and_display(event_url)


/opt/anaconda3/lib/python3.12/site-packages/pyppeteer/util.py:29: RuntimeWarning: coroutine 'scrape_highape_events' was never awaited
  gc.collect()


TimeoutError: Navigation Timeout Exceeded: 30000 ms exceeded.

In [15]:
import nest_asyncio
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup
import pandas as pd
import re
import html
from IPython.display import display

nest_asyncio.apply()

# === STEP 1: Load HTML from URL with increased timeout ===
async def get_page_content(url):
    browser = await launch(headless=True, args=['--no-sandbox', '--disable-setuid-sandbox'])
    page = await browser.newPage()
    await page.setUserAgent("Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36")
    try:
        await page.goto(url, {"waitUntil": "networkidle2", "timeout": 30000})  # 60s timeout
        content = await page.content()
    except Exception as e:
        print(f"❌ Failed to load page: {e}")
        content = ""
    await browser.close()
    return content

# === STEP 2: Parse event details from HTML ===
def parse_event_page(html_content):
    soup = BeautifulSoup(html_content, "html.parser")

    def clean_text(el):
        return html.unescape(el.get_text(strip=True)) if el else ""

    def extract_json_ld():
        script_tag = soup.find("script", type="application/ld+json")
        if not script_tag:
            return {}
        try:
            import json
            return json.loads(script_tag.string)
        except:
            return {}

    ld_json = extract_json_ld()

    return {
        "eventName": ld_json.get("name", ""),
        "eventDescription": BeautifulSoup(ld_json.get("description", ""), "html.parser").prettify(),
        "eventDateAndTime": ld_json.get("startDate", ""),
        "eventDuration": "",
        "venue": {
            "venueName": ld_json.get("location", {}).get("name", ""),
            "locality": "",
            "address": ld_json.get("location", {}).get("address", {}).get("streetAddress", ""),
            "city": ld_json.get("location", {}).get("address", {}).get("addressLocality", ""),
            "state": ld_json.get("location", {}).get("address", {}).get("addressRegion", ""),
            "zipcode": ld_json.get("location", {}).get("address", {}).get("postalCode", ""),
            "geolocation": {
                "lat": ld_json.get("location", {}).get("geo", {}).get("latitude", ""),
                "lon": ld_json.get("location", {}).get("geo", {}).get("longitude", "")
            },
            "layout": ""
        },
        "highlightImageLinks": [ld_json.get("image", "")] if ld_json.get("image") else [],
        "galleryImageLinks": [],
        "ticketAmount": "",
        "ticketLink": "",
        "supportedLanguages": [],
        "category": "",
        "subCategory": "",
        "eventType": "Event",
        "eventFeatures": {},
        "artists": [],
        "sharableEventOgImageLink": ld_json.get("image", ""),
        "attendeesCount": "",
        "likesCount": "",
        "joinChatDetails": {
            "joinChatLink": "",
            "provider": "",
            "isEnabled": False
        },
        "policyAndConditions": [],
        "frequentlyAskedQuestions": []
    }

# === STEP 3: Run and display result ===
async def scrape_event_and_display(url):
    html_content = await get_page_content(url)
    if not html_content:
        print("⚠️ Empty page. Try checking your network or site structure.")
        return
    event_data = parse_event_page(html_content)
    df = pd.json_normalize(event_data, sep='.')
    pd.set_option("display.max_colwidth", None)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.expand_frame_repr", False)
    display(df)

# === STEP 4: URL to test ===
url = "https://highape.com/bangalore/events/match-me-please-5ewaeRxjwE?utm_source=allevents_list"
await scrape_event_and_display(url)


❌ Failed to load page: Navigation Timeout Exceeded: 30000 ms exceeded.
⚠️ Empty page. Try checking your network or site structure.


In [3]:
import json
import uuid
import requests
from bs4 import BeautifulSoup
import pandas as pd

# === Step 1: Load HTML from live event link ===
event_url = "https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY"
response = requests.get(event_url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "html.parser")

# === Step 2: Parse JSON-LD structured data ===
schema_tag = soup.find("script", type="application/ld+json")
schema_json = json.loads(schema_tag.string)
schema = schema_json[0] if isinstance(schema_json, list) else schema_json

# === Step 3: Extract extra info like coordinates ===
lat_tag = soup.find("input", {"id": "venue_lat"})
lon_tag = soup.find("input", {"id": "venue_lng"})
lat = float(lat_tag['value']) if lat_tag else ''
lon = float(lon_tag['value']) if lon_tag else ''

# === Step 4: Prepare output ===
event = {
    'externalEventLink': schema.get('url', ''),
    'eventId': str(uuid.uuid4()),
    'plurEventLink': '',
    'eventName': schema.get('name', ''),
    'eventDescription': schema.get('description', ''),
    'eventDateAndTime': schema.get('startDate', ''),
    'eventDuration': '',
    'venueName': schema.get('location', {}).get('name', ''),
    'locality': '',
    'address': schema.get('location', {}).get('address', ''),
    'zipcode': schema.get('location', {}).get('address', '').split()[-1],
    'lat': lat,
    'lon': lon,
    'ticketAmount': schema.get('offers', [{}])[0].get('price', '') if schema.get('offers') else '',
    'guestlistInfo': '',
    'ticketLink': schema.get('offers', [{}])[0].get('url', '') if schema.get('offers') else '',
    'supportedLanguages': '',
    'category': '',
    'recurrence': '',
    'attendeesCount': '',
    'joinChatDetails': '',
    'policyAndConditions': '',
    'frequentlyAskedQuestions': '',
    'city': '',
    'state': '',
    'geolocation': '',
    'isRecurring': 'No',
    'pattern': '',
    'eventType': schema.get('@type', ''),
    'subCategory': '',
    'venueId': '',
    'foodAvailable': '',
    'smokingAllowed': '',
    'wheelchairAccess': '',
    'parkingAvailable': '',
    'supportAvailable': '',
    'layout': '',
    'petFriendly': False,
    'alcoholServed': '',
    'minimumAge': '',
    'ticketsAtVenue': '',
    'prohibitedItemList': '',
    'washroomAvailable': '',
    'onlineMenu': '',
    'happyHours': '',
    'danceFloorAvailable': '',
    'poolAvailable': '',
    'isActivityAvailable': '',
    'artists': '',
    'highlightImages': schema.get('image', ''),
    'highlightVideos': '',
    'galleryImages': '',
    'galleryVideos': '',
    'kidFriendly': False,
    'seatingArrangement': '',
    'entryAllowedFor': '',
    'tags': ''
}

# === Step 5: Display output ===
df = pd.DataFrame([event])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
print(df.to_markdown(index=False))


| externalEventLink                                                                         | eventId                              | plurEventLink   | eventName                           | eventDescription                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | eventDateAndTime    | eventDuration   | venueName          | locality   | address                                                                          | zipcode   |     lat |    lon |   ticketAmount | guestlistInfo   | ticketLink                                                                   

In [24]:
!pip install ace_tools

In [5]:
import json
import uuid
import requests
from bs4 import BeautifulSoup
import pandas as pd

# === Step 1: Load HTML from live event link ===
event_url = "https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY"
response = requests.get(event_url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "html.parser")

# === Step 2: Parse JSON-LD structured data ===
schema_tag = soup.find("script", type="application/ld+json")
schema_json = json.loads(schema_tag.string)
schema = schema_json[0] if isinstance(schema_json, list) else schema_json

# === Step 3: Extract extra info like coordinates ===
lat_tag = soup.find("input", {"id": "venue_lat"})
lon_tag = soup.find("input", {"id": "venue_lng"})
lat = float(lat_tag['value']) if lat_tag else ''
lon = float(lon_tag['value']) if lon_tag else ''

# === Step 4: Prepare output ===
event = {
    'externalEventLink': schema.get('url', ''),
    'eventId': str(uuid.uuid4()),
    'plurEventLink': '',
    'eventName': schema.get('name', ''),
    'eventDescription': schema.get('description', ''),
    'eventDateAndTime': schema.get('startDate', ''),
    'eventDuration': '',
    'venueName': schema.get('location', {}).get('name', ''),
    'locality': '',
    'address': schema.get('location', {}).get('address', ''),
    'zipcode': schema.get('location', {}).get('address', '').split()[-1],
    'lat': lat,
    'lon': lon,
    'ticketAmount': schema.get('offers', [{}])[0].get('price', '') if schema.get('offers') else '',
    'guestlistInfo': '',
    'ticketLink': schema.get('offers', [{}])[0].get('url', '') if schema.get('offers') else '',
    'supportedLanguages': '',
    'category': '',
    'recurrence': '',
    'attendeesCount': '',
    'joinChatDetails': '',
    'policyAndConditions': '',
    'frequentlyAskedQuestions': '',
    'city': '',
    'state': '',
    'geolocation': '',
    'isRecurring': 'No',
    'pattern': '',
    'eventType': schema.get('@type', ''),
    'subCategory': '',
    'venueId': '',
    'foodAvailable': '',
    'smokingAllowed': '',
    'wheelchairAccess': '',
    'parkingAvailable': '',
    'supportAvailable': '',
    'layout': '',
    'petFriendly': False,
    'alcoholServed': '',
    'minimumAge': '',
    'ticketsAtVenue': '',
    'prohibitedItemList': '',
    'washroomAvailable': '',
    'onlineMenu': '',
    'happyHours': '',
    'danceFloorAvailable': '',
    'poolAvailable': '',
    'isActivityAvailable': '',
    'artists': '',
    'highlightImages': schema.get('image', ''),
    'highlightVideos': '',
    'galleryImages': '',
    'galleryVideos': '',
    'kidFriendly': False,
    'seatingArrangement': '',
    'entryAllowedFor': '',
    'tags': ''
}

# === Step 5: Display output ===
df = pd.DataFrame([event])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
print(df.to_markdown(index=False))


| externalEventLink                                                                         | eventId                              | plurEventLink   | eventName                           | eventDescription                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | eventDateAndTime    | eventDuration   | venueName          | locality   | address                                                                          | zipcode   |     lat |    lon |   ticketAmount | guestlistInfo   | ticketLink                                                                   

In [9]:
import json
import uuid
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, HTML

# === Step 1: Load HTML from live event link ===
event_url = "https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY"
response = requests.get(event_url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "html.parser")

# === Step 2: Parse JSON-LD structured data ===
schema_tag = soup.find("script", type="application/ld+json")
schema_json = json.loads(schema_tag.string)
schema = schema_json[0] if isinstance(schema_json, list) else schema_json

# === Step 3: Extract extra info like coordinates ===
lat_tag = soup.find("input", {"id": "venue_lat"})
lon_tag = soup.find("input", {"id": "venue_lng"})
lat = float(lat_tag['value']) if lat_tag else ''
lon = float(lon_tag['value']) if lon_tag else ''

# === Step 4: Prepare output ===
event = {
    'externalEventLink': schema.get('url', ''),
    'eventId': str(uuid.uuid4()),
    'plurEventLink': '',
    'eventName': schema.get('name', ''),
    'eventDescription': schema.get('description', ''),
    'eventDateAndTime': schema.get('startDate', ''),
    'eventDuration': '',
    'venueName': schema.get('location', {}).get('name', ''),
    'locality': '',
    'address': schema.get('location', {}).get('address', ''),
    'zipcode': schema.get('location', {}).get('address', '').split()[-1],
    'lat': lat,
    'lon': lon,
    'ticketAmount': schema.get('offers', [{}])[0].get('price', '') if schema.get('offers') else '',
    'guestlistInfo': '',
    'ticketLink': schema.get('offers', [{}])[0].get('url', '') if schema.get('offers') else '',
    'supportedLanguages': '',
    'category': '',
    'recurrence': '',
    'attendeesCount': '',
    'joinChatDetails': '',
    'policyAndConditions': '',
    'frequentlyAskedQuestions': '',
    'city': '',
    'state': '',
    'geolocation': '',
    'isRecurring': 'No',
    'pattern': '',
    'eventType': schema.get('@type', ''),
    'subCategory': '',
    'venueId': '',
    'foodAvailable': '',
    'smokingAllowed': '',
    'wheelchairAccess': '',
    'parkingAvailable': '',
    'supportAvailable': '',
    'layout': '',
    'petFriendly': False,
    'alcoholServed': '',
    'minimumAge': '',
    'ticketsAtVenue': '',
    'prohibitedItemList': '',
    'washroomAvailable': '',
    'onlineMenu': '',
    'happyHours': '',
    'danceFloorAvailable': '',
    'poolAvailable': '',
    'isActivityAvailable': '',
    'artists': '',
    'highlightImages': schema.get('image', ''),
    'highlightVideos': '',
    'galleryImages': '',
    'galleryVideos': '',
    'kidFriendly': False,
    'seatingArrangement': '',
    'entryAllowedFor': '',
    'tags': '',
    'rawHtml': response.text
}

# === Step 5: Display extracted schema in readable JSON format ===
print("\n========== Parsed JSON-LD Schema ==========")
print(json.dumps(schema, indent=2))

# === Step 6: Display dataframe ===
df = pd.DataFrame([event])
tools.display_dataframe_to_user(name="HighApe Event Extracted", dataframe=df)

# === Step 7: Pretty-print raw HTML in viewer ===
display(HTML(f"<pre>{event['rawHtml'][:10000]}</pre>"))



========== Parsed JSON-LD Schema ==========
{
  "@context": "http://schema.org",
  "@type": "Event",
  "name": "Jam Eve - A Jamming Show \u00d7 Open Mic",
  "image": "https://res.cloudinary.com/https-highape-com/image/upload/q_auto:eco,f_auto,h_530/v1749203817/q2kvkuiu2odr3pb1hx3z.jpg",
  "url": "https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY",
  "startDate": "2025-06-07 18:30:00",
  "endDate": "2025-06-28 21:30:00",
  "doorTime": "2025-06-07 18:30:00",
  "description": "Highlights High-energy jam session and open mic Platform for singers, musicians, poets &amp; performers A creative, welcoming space for self-expression Connect with like-minded artists and music lovers Discover talent, build friendships, and create memories Perfect blend of art, music, and vibe Step into a night of rhythm, soul, and spontaneity at Zyvas Jam-Eve &amp; Open Mic a celebration of self-expression and shared vibes. Whether youre a seasoned performer or just looking

NameError: name 'tools' is not defined

In [11]:
import json
import uuid
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, HTML

# === Step 1: Load HTML from live event link ===
event_url = "https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY"
response = requests.get(event_url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "html.parser")

# === Step 2: Parse JSON-LD structured data ===
schema_tag = soup.find("script", type="application/ld+json")
schema_json = json.loads(schema_tag.string)
schema = schema_json[0] if isinstance(schema_json, list) else schema_json

# === Step 3: Extract extra info like coordinates ===
lat_tag = soup.find("input", {"id": "venue_lat"})
lon_tag = soup.find("input", {"id": "venue_lng"})
lat = float(lat_tag['value']) if lat_tag else ''
lon = float(lon_tag['value']) if lon_tag else ''

# === Step 4: Prepare output ===
event = {
    'externalEventLink': schema.get('url', ''),
    'eventId': str(uuid.uuid4()),
    'plurEventLink': '',
    'eventName': schema.get('name', ''),
    'eventDescription': schema.get('description', ''),
    'eventDateAndTime': schema.get('startDate', ''),
    'eventDuration': '',
    'venueName': schema.get('location', {}).get('name', ''),
    'locality': '',
    'address': schema.get('location', {}).get('address', ''),
    'zipcode': schema.get('location', {}).get('address', '').split()[-1],
    'lat': lat,
    'lon': lon,
    'ticketAmount': schema.get('offers', [{}])[0].get('price', '') if schema.get('offers') else '',
    'guestlistInfo': '',
    'ticketLink': schema.get('offers', [{}])[0].get('url', '') if schema.get('offers') else '',
    'supportedLanguages': '',
    'category': '',
    'recurrence': '',
    'attendeesCount': '',
    'joinChatDetails': '',
    'policyAndConditions': '',
    'frequentlyAskedQuestions': '',
    'city': '',
    'state': '',
    'geolocation': '',
    'isRecurring': 'No',
    'pattern': '',
    'eventType': schema.get('@type', ''),
    'subCategory': '',
    'venueId': '',
    'foodAvailable': '',
    'smokingAllowed': '',
    'wheelchairAccess': '',
    'parkingAvailable': '',
    'supportAvailable': '',
    'layout': '',
    'petFriendly': False,
    'alcoholServed': '',
    'minimumAge': '',
    'ticketsAtVenue': '',
    'prohibitedItemList': '',
    'washroomAvailable': '',
    'onlineMenu': '',
    'happyHours': '',
    'danceFloorAvailable': '',
    'poolAvailable': '',
    'isActivityAvailable': '',
    'artists': '',
    'highlightImages': schema.get('image', ''),
    'highlightVideos': '',
    'galleryImages': '',
    'galleryVideos': '',
    'kidFriendly': False,
    'seatingArrangement': '',
    'entryAllowedFor': '',
    'tags': '',
    'rawHtml': response.text
}

# === Step 5: Display extracted schema in readable JSON format ===
print("\n========== Parsed JSON-LD Schema ==========")
print(json.dumps(schema, indent=2))

# === Step 6: Display dataframe ===
df = pd.DataFrame([event])
display(df)

# === Step 7: Pretty-print raw HTML in viewer ===
display(HTML(f"<pre>{event['rawHtml'][:10000]}</pre>"))



========== Parsed JSON-LD Schema ==========
{
  "@context": "http://schema.org",
  "@type": "Event",
  "name": "Jam Eve - A Jamming Show \u00d7 Open Mic",
  "image": "https://res.cloudinary.com/https-highape-com/image/upload/q_auto:eco,f_auto,h_530/v1749203817/q2kvkuiu2odr3pb1hx3z.jpg",
  "url": "https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY",
  "startDate": "2025-06-07 18:30:00",
  "endDate": "2025-06-28 21:30:00",
  "doorTime": "2025-06-07 18:30:00",
  "description": "Highlights High-energy jam session and open mic Platform for singers, musicians, poets &amp; performers A creative, welcoming space for self-expression Connect with like-minded artists and music lovers Discover talent, build friendships, and create memories Perfect blend of art, music, and vibe Step into a night of rhythm, soul, and spontaneity at Zyvas Jam-Eve &amp; Open Mic a celebration of self-expression and shared vibes. Whether youre a seasoned performer or just looking

externalEventLink  \
0  https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY   

                                eventId plurEventLink  \
0  173d2318-13d3-466a-a8cd-306e7444deab                 

                             eventName  \
0  Jam Eve - A Jamming Show × Open Mic   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               eventDescription  \
0  Highlights High-energy jam session and open mic Platform for singers, musicians, poets &amp; performers A creative, welcoming space for self-expression Connect with like-minded artists and music lovers Discover talent, build friendships, and create memories Perfect blend of art, music, and vibe Step into a night of rhythm, soul, and spontaneity at Zyvas Jam-Eve &amp; Open Mic a celebration of self-expression and shared vibes. Whether youre a seasoned performer or just looking to hum along, this space is   

      eventDateAndTime eventDuration           venueName locality  \
0  2025-06-07 18:30:00                Catch Up Bangalore            

                                                                            address  \
0  253, 17th Cross Road, 6th Sector, HSR Layout, Bengaluru, Karnataka 560102, India   

  zipcode        lat        lon ticketAmount guestlistInfo  \
0   India  12.912582  77.633017          199                 

                                                                                                                              ticketLink  \
0  https://tickets.highape.com/?event_name=jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY&amp;city=bangalore&amp;module_type=event   

  supportedLanguages category recurrence attendeesCount joinChatDetails  \
0                                                                         

  policyAndConditions frequentlyAskedQuestions city state geolocation  \
0                                                                       

  isRecurring pattern eventType subCategory venueId foodAvailable  \
0          No             Event                                     

  smokingAllowed wheelchairAccess parkingAvailable supportAvailable layout  \
0                                                                            

   petFriendly alcoholServed minimumAge ticketsAtVenue prohibitedItemList  \
0        False                                                              

  washroomAvailable onlineMenu happyHours danceFloorAvailable poolAvailable  \
0                                                                             

  isActivityAvailable artists  \
0                               

                                                                                                          highlightImages  \
0  https://res.cloudinary.com/https-highape-com/image/upload/q_auto:eco,f_auto,h_530/v1749203817/q2kvkuiu2odr3pb1hx3z.jpg   

  highlightVideos galleryImages galleryVideos  kidFriendly seatingArrangement  \
0                                                    False                      

  entryAllowedFor tags  \
0                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [13]:
import json
import uuid
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, HTML
from pathlib import Path

# === Step 1: Load HTML from live event link ===
event_url = "https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY"
response = requests.get(event_url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "html.parser")

# === Step 2: Parse JSON-LD structured data ===
schema_tag = soup.find("script", type="application/ld+json")
schema_json = json.loads(schema_tag.string)
schema = schema_json[0] if isinstance(schema_json, list) else schema_json

# === Step 3: Extract extra info like coordinates ===
lat_tag = soup.find("input", {"id": "venue_lat"})
lon_tag = soup.find("input", {"id": "venue_lng"})
lat = float(lat_tag['value']) if lat_tag else ''
lon = float(lon_tag['value']) if lon_tag else ''

# === Step 4: Prepare output ===
event = {
    'externalEventLink': schema.get('url', ''),
    'eventId': str(uuid.uuid4()),
    'plurEventLink': '',
    'eventName': schema.get('name', ''),
    'eventDescription': schema.get('description', ''),
    'eventDateAndTime': schema.get('startDate', ''),
    'eventDuration': '',
    'venueName': schema.get('location', {}).get('name', ''),
    'locality': '',
    'address': schema.get('location', {}).get('address', ''),
    'zipcode': schema.get('location', {}).get('address', '').split()[-1],
    'lat': lat,
    'lon': lon,
    'ticketAmount': schema.get('offers', [{}])[0].get('price', '') if schema.get('offers') else '',
    'guestlistInfo': '',
    'ticketLink': schema.get('offers', [{}])[0].get('url', '') if schema.get('offers') else '',
    'supportedLanguages': '',
    'category': '',
    'recurrence': '',
    'attendeesCount': '',
    'joinChatDetails': '',
    'policyAndConditions': '',
    'frequentlyAskedQuestions': '',
    'city': '',
    'state': '',
    'geolocation': '',
    'isRecurring': 'No',
    'pattern': '',
    'eventType': schema.get('@type', ''),
    'subCategory': '',
    'venueId': '',
    'foodAvailable': '',
    'smokingAllowed': '',
    'wheelchairAccess': '',
    'parkingAvailable': '',
    'supportAvailable': '',
    'layout': '',
    'petFriendly': False,
    'alcoholServed': '',
    'minimumAge': '',
    'ticketsAtVenue': '',
    'prohibitedItemList': '',
    'washroomAvailable': '',
    'onlineMenu': '',
    'happyHours': '',
    'danceFloorAvailable': '',
    'poolAvailable': '',
    'isActivityAvailable': '',
    'artists': '',
    'highlightImages': schema.get('image', ''),
    'highlightVideos': '',
    'galleryImages': '',
    'galleryVideos': '',
    'kidFriendly': False,
    'seatingArrangement': '',
    'entryAllowedFor': '',
    'tags': '',
    'rawHtml': response.text
}

# === Step 5: Display extracted schema in readable JSON format ===
print("\n========== Parsed JSON-LD Schema ==========")
print(json.dumps(schema, indent=2))

# === Step 6: Display dataframe ===
df = pd.DataFrame([event])
display(df)

# === Step 7: Save to CSV for testing ===
csv_path = Path("/mnt/data/highape_event_test.csv")
df.to_csv(csv_path, index=False)
print(f"✅ CSV saved to: {csv_path}")

# === Step 8: Pretty-print raw HTML in viewer ===
display(HTML(f"<pre>{event['rawHtml'][:10000]}</pre>"))



========== Parsed JSON-LD Schema ==========
{
  "@context": "http://schema.org",
  "@type": "Event",
  "name": "Jam Eve - A Jamming Show \u00d7 Open Mic",
  "image": "https://res.cloudinary.com/https-highape-com/image/upload/q_auto:eco,f_auto,h_530/v1749203817/q2kvkuiu2odr3pb1hx3z.jpg",
  "url": "https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY",
  "startDate": "2025-06-07 18:30:00",
  "endDate": "2025-06-28 21:30:00",
  "doorTime": "2025-06-07 18:30:00",
  "description": "Highlights High-energy jam session and open mic Platform for singers, musicians, poets &amp; performers A creative, welcoming space for self-expression Connect with like-minded artists and music lovers Discover talent, build friendships, and create memories Perfect blend of art, music, and vibe Step into a night of rhythm, soul, and spontaneity at Zyvas Jam-Eve &amp; Open Mic a celebration of self-expression and shared vibes. Whether youre a seasoned performer or just looking

externalEventLink  \
0  https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY   

                                eventId plurEventLink  \
0  d4d7f670-0b6b-4233-a889-6a915493c6ca                 

                             eventName  \
0  Jam Eve - A Jamming Show × Open Mic   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               eventDescription  \
0  Highlights High-energy jam session and open mic Platform for singers, musicians, poets &amp; performers A creative, welcoming space for self-expression Connect with like-minded artists and music lovers Discover talent, build friendships, and create memories Perfect blend of art, music, and vibe Step into a night of rhythm, soul, and spontaneity at Zyvas Jam-Eve &amp; Open Mic a celebration of self-expression and shared vibes. Whether youre a seasoned performer or just looking to hum along, this space is   

      eventDateAndTime eventDuration           venueName locality  \
0  2025-06-07 18:30:00                Catch Up Bangalore            

                                                                            address  \
0  253, 17th Cross Road, 6th Sector, HSR Layout, Bengaluru, Karnataka 560102, India   

  zipcode        lat        lon ticketAmount guestlistInfo  \
0   India  12.912582  77.633017          199                 

                                                                                                                              ticketLink  \
0  https://tickets.highape.com/?event_name=jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY&amp;city=bangalore&amp;module_type=event   

  supportedLanguages category recurrence attendeesCount joinChatDetails  \
0                                                                         

  policyAndConditions frequentlyAskedQuestions city state geolocation  \
0                                                                       

  isRecurring pattern eventType subCategory venueId foodAvailable  \
0          No             Event                                     

  smokingAllowed wheelchairAccess parkingAvailable supportAvailable layout  \
0                                                                            

   petFriendly alcoholServed minimumAge ticketsAtVenue prohibitedItemList  \
0        False                                                              

  washroomAvailable onlineMenu happyHours danceFloorAvailable poolAvailable  \
0                                                                             

  isActivityAvailable artists  \
0                               

                                                                                                          highlightImages  \
0  https://res.cloudinary.com/https-highape-com/image/upload/q_auto:eco,f_auto,h_530/v1749203817/q2kvkuiu2odr3pb1hx3z.jpg   

  highlightVideos galleryImages galleryVideos  kidFriendly seatingArrangement  \
0                                                    False                      

  entryAllowedFor tags  \
0                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

OSError: Cannot save file into a non-existent directory: '/mnt/data'

In [15]:
import json
import uuid
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display, HTML
from pathlib import Path
import os

# === Step 1: Load HTML from live event link ===
event_url = "https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY"
response = requests.get(event_url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "html.parser")

# === Step 2: Parse JSON-LD structured data ===
schema_tag = soup.find("script", type="application/ld+json")
schema_json = json.loads(schema_tag.string)
schema = schema_json[0] if isinstance(schema_json, list) else schema_json

# === Step 3: Extract extra info like coordinates ===
lat_tag = soup.find("input", {"id": "venue_lat"})
lon_tag = soup.find("input", {"id": "venue_lng"})
lat = float(lat_tag['value']) if lat_tag else ''
lon = float(lon_tag['value']) if lon_tag else ''

# === Step 4: Prepare output ===
event = {
    'externalEventLink': schema.get('url', ''),
    'eventId': str(uuid.uuid4()),
    'plurEventLink': '',
    'eventName': schema.get('name', ''),
    'eventDescription': schema.get('description', ''),
    'eventDateAndTime': schema.get('startDate', ''),
    'eventDuration': '',
    'venueName': schema.get('location', {}).get('name', ''),
    'locality': '',
    'address': schema.get('location', {}).get('address', ''),
    'zipcode': schema.get('location', {}).get('address', '').split()[-1],
    'lat': lat,
    'lon': lon,
    'ticketAmount': schema.get('offers', [{}])[0].get('price', '') if schema.get('offers') else '',
    'guestlistInfo': '',
    'ticketLink': schema.get('offers', [{}])[0].get('url', '') if schema.get('offers') else '',
    'supportedLanguages': '',
    'category': '',
    'recurrence': '',
    'attendeesCount': '',
    'joinChatDetails': '',
    'policyAndConditions': '',
    'frequentlyAskedQuestions': '',
    'city': '',
    'state': '',
    'geolocation': '',
    'isRecurring': 'No',
    'pattern': '',
    'eventType': schema.get('@type', ''),
    'subCategory': '',
    'venueId': '',
    'foodAvailable': '',
    'smokingAllowed': '',
    'wheelchairAccess': '',
    'parkingAvailable': '',
    'supportAvailable': '',
    'layout': '',
    'petFriendly': False,
    'alcoholServed': '',
    'minimumAge': '',
    'ticketsAtVenue': '',
    'prohibitedItemList': '',
    'washroomAvailable': '',
    'onlineMenu': '',
    'happyHours': '',
    'danceFloorAvailable': '',
    'poolAvailable': '',
    'isActivityAvailable': '',
    'artists': '',
    'highlightImages': schema.get('image', ''),
    'highlightVideos': '',
    'galleryImages': '',
    'galleryVideos': '',
    'kidFriendly': False,
    'seatingArrangement': '',
    'entryAllowedFor': '',
    'tags': '',
    'rawHtml': response.text
}

# === Step 5: Display extracted schema in readable JSON format ===
print("\n========== Parsed JSON-LD Schema ==========")
print(json.dumps(schema, indent=2))

# === Step 6: Display dataframe ===
df = pd.DataFrame([event])
display(df)

# === Step 7: Save to CSV for testing ===
os.makedirs("exports", exist_ok=True)
csv_path = Path("exports/highape_event_test.csv")
df.to_csv(csv_path, index=False)
print(f"✅ CSV saved to: {csv_path.resolve()}")

# === Step 8: Pretty-print raw HTML in viewer ===
display(HTML(f"<pre>{event['rawHtml'][:10000]}</pre>"))



========== Parsed JSON-LD Schema ==========
{
  "@context": "http://schema.org",
  "@type": "Event",
  "name": "Jam Eve - A Jamming Show \u00d7 Open Mic",
  "image": "https://res.cloudinary.com/https-highape-com/image/upload/q_auto:eco,f_auto,h_530/v1749203817/q2kvkuiu2odr3pb1hx3z.jpg",
  "url": "https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY",
  "startDate": "2025-06-07 18:30:00",
  "endDate": "2025-06-28 21:30:00",
  "doorTime": "2025-06-07 18:30:00",
  "description": "Highlights High-energy jam session and open mic Platform for singers, musicians, poets &amp; performers A creative, welcoming space for self-expression Connect with like-minded artists and music lovers Discover talent, build friendships, and create memories Perfect blend of art, music, and vibe Step into a night of rhythm, soul, and spontaneity at Zyvas Jam-Eve &amp; Open Mic a celebration of self-expression and shared vibes. Whether youre a seasoned performer or just looking

externalEventLink  \
0  https://highape.com/bangalore/events/jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY   

                                eventId plurEventLink  \
0  0f67b873-df1f-44a2-af4a-9fd3be93282a                 

                             eventName  \
0  Jam Eve - A Jamming Show × Open Mic   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               eventDescription  \
0  Highlights High-energy jam session and open mic Platform for singers, musicians, poets &amp; performers A creative, welcoming space for self-expression Connect with like-minded artists and music lovers Discover talent, build friendships, and create memories Perfect blend of art, music, and vibe Step into a night of rhythm, soul, and spontaneity at Zyvas Jam-Eve &amp; Open Mic a celebration of self-expression and shared vibes. Whether youre a seasoned performer or just looking to hum along, this space is   

      eventDateAndTime eventDuration           venueName locality  \
0  2025-06-07 18:30:00                Catch Up Bangalore            

                                                                            address  \
0  253, 17th Cross Road, 6th Sector, HSR Layout, Bengaluru, Karnataka 560102, India   

  zipcode        lat        lon ticketAmount guestlistInfo  \
0   India  12.912582  77.633017          199                 

                                                                                                                              ticketLink  \
0  https://tickets.highape.com/?event_name=jam-eve-bangalore-a-jamming-show-open-mic-a7kjyp7dwY&amp;city=bangalore&amp;module_type=event   

  supportedLanguages category recurrence attendeesCount joinChatDetails  \
0                                                                         

  policyAndConditions frequentlyAskedQuestions city state geolocation  \
0                                                                       

  isRecurring pattern eventType subCategory venueId foodAvailable  \
0          No             Event                                     

  smokingAllowed wheelchairAccess parkingAvailable supportAvailable layout  \
0                                                                            

   petFriendly alcoholServed minimumAge ticketsAtVenue prohibitedItemList  \
0        False                                                              

  washroomAvailable onlineMenu happyHours danceFloorAvailable poolAvailable  \
0                                                                             

  isActivityAvailable artists  \
0                               

                                                                                                          highlightImages  \
0  https://res.cloudinary.com/https-highape-com/image/upload/q_auto:eco,f_auto,h_530/v1749203817/q2kvkuiu2odr3pb1hx3z.jpg   

  highlightVideos galleryImages galleryVideos  kidFriendly seatingArrangement  \
0                                                    False                      

  entryAllowedFor tags  \
0                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

✅ CSV saved to: /Users/shiv/Documents/Jupyter/HighApe/exports/highape_event_test.csv
